In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('train.csv', encoding='latin-1', index_col=0)

print(df.shape)

(31962, 3)


In [3]:
df.head()

,id,label,tweet
1,1,0,father dysfunctional selfish drags ki...
2,2,0,thanks lyft credit cant use cause dont of...
3,3,0,Bihday majesty
4,4,0,Model love u take u time ur
5,5,0,Factsguide society now motivation


In [4]:
df = df.dropna()

## Data Splitting

In [5]:
X_train = df['tweet']
y_train = df['label']

In [6]:
from sklearn.feature_selection import chi2, SelectKBest

from sklearn.pipeline import Pipeline, FeatureUnion

from sklearn.preprocessing import Imputer
from sklearn.feature_extraction.text import HashingVectorizer, TfidfTransformer

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

from sklearn.preprocessing import MaxAbsScaler

TOKENS_ALPHANUMERIC = '[A-Za-z0-9]+(?=\\s+)'

In [7]:
from features.SparseInteractions import SparseInteractions

In [9]:
# ignore deprecation warnings in sklearn
import warnings
warnings.filterwarnings("ignore")

# setting a reasonable number of features before adding interactions
chi_k = 300

# creating the pipeline object
pl = Pipeline([ ('text_features', Pipeline([
                    ('vectorizer', HashingVectorizer(token_pattern=TOKENS_ALPHANUMERIC,
                                                     non_negative=True, norm=None, binary=False,
                                                     ngram_range=(1, 2))),
                    ('tfidf', TfidfTransformer()),
                    ('dim_red', SelectKBest(chi2, chi_k))
              ])),
        ('int', SparseInteractions(degree=2)),
        #('scale', MaxAbsScaler()),
        ('clf', RandomForestClassifier(n_estimators=15))
    ])

# fitting the pipeline to our training data
pl.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('text_features', Pipeline(memory=None,
     steps=[('vectorizer', HashingVectorizer(alternate_sign=True, analyzer='word', binary=False,
         decode_error='strict', dtype=<class 'numpy.float64'>,
         encoding='utf-8', input='content', lowercase=True,
         n_features=1048576, ngra...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [10]:
# ignore deprecation warnings in sklearn
import warnings
warnings.filterwarnings("ignore")

from sklearn.metrics import classification_report

accuracy = pl.score(X_train, y_train)
print("\nAccuracy on test dataset: ", accuracy)

y_pred = pl.predict(X_train)
print(classification_report(y_train, y_pred))


Accuracy on test dataset:  0.978443151242
             precision    recall  f1-score   support

          0       0.98      1.00      0.99     29720
          1       0.99      0.70      0.82      2242

avg / total       0.98      0.98      0.98     31962



## Test Accuracy

In [13]:
test = pd.read_csv('test.csv', encoding='latin-1', index_col=0)
X_test = test['tweet']

# fitting the pipeline to our training data
vectorizer = HashingVectorizer(token_pattern=TOKENS_ALPHANUMERIC, non_negative=True, norm=None, binary=False, ngram_range=(1, 2))
vectorizer.transform(X_test)

tfidf = TfidfTransformer()
tfidf.transform(X_test)

ValueError: could not convert string to float: 'My song  glad free download shoegaze newmusic newsong'

In [12]:
# Making predictions
predictions = pl.predict(X_test)
prob = pl.predict_proba(X_test)[:,1]
dt = {'label':predictions, 'prob':prob}
#temp = pd.DataFrame(np.array(predictions).reshape(100128,104))
# Formatting correctly in new DataFrame: prediction_df
prediction_df = pd.DataFrame(data=dt, index=test.id)


# Saving prediction_df to csv called "predictions.csv"
prediction_df.to_csv("predictions.csv")